In [1]:
# from LMlstm import SIFT_LSTM
from loadLabels import load_labels
import torch
import torch.nn as nn
from loadfeat import load_FeatfromCSV,split_train_val
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset,DataLoader
import torch.nn.functional as F
import math
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [2]:

def load_input_data(dataset,labels):

    input_data=[]
    input_labels=[]
    for foldername in list(dataset.keys()):
        
        data=dataset[foldername]
        label=labels[foldername]
        
        tensordata=torch.tensor(data)
        tensorlabel=torch.tensor(label)

        input_data.append(tensordata)
        input_labels.append(tensorlabel)

    return input_data,input_labels

In [3]:
AU_dir = '/home/ubuntu/gxy/535proj/FacialLMextraction/extractedLM/AUs'
labels_dir = "/home/ubuntu/gxy/535proj/dataset/SEWAv02"
lang='C1'
dataset=load_FeatfromCSV(AU_dir,lang)
print('finish loading dataset')


len(csv_files)=102
finish loading dataset


In [4]:
train_data, val_data = split_train_val(dataset)
print('finish split_train_val dataset')
    # Output the lengths of training and validation sets
print("Training set length:", len(train_data))
print(f'Length of the first element in train_data: {len(list(train_data.values())[0])}')
print("Validation set length:", len(val_data))

finish split_train_val dataset
Training set length: 81
Length of the first element in train_data: 801
Validation set length: 21


In [5]:
labels=load_labels(root_dir=labels_dir,language='C1',type='Arousal',suffix='AV_Aligned')
print('finish loading labels')
train_inputs,train_labels=load_input_data(train_data,labels)
print('finish load_training_data')
val_inputs,val_labels=load_input_data(val_data,labels)
print('finish load_val_data')

len(files)=102
processing SSL_C1_S194_P388_VC1_005501_005901_Arousal_AV_Aligned
processing SAH_C1_S029_P058_VC1_006573_007472_Arousal_AV_Aligned
processing SAL_C1_S001_P002_VC1_000921_001732_Arousal_AV_Aligned
processing SSD_C1_S023_P045_VC1_001801_002201_Arousal_AV_Aligned
processing SAH_C1_S009_P018_VC1_004321_005367_Arousal_AV_Aligned
processing SVL_C1_S006_P012_VC1_007601_008701_Arousal_AV_Aligned
processing SSD_C1_S026_P052_VC1_001101_001901_Arousal_AV_Aligned
processing SSD_C1_S022_P044_VC1_007401_008001_Arousal_AV_Aligned
processing SVH_C1_S027_P054_VC1_001601_002101_Arousal_AV_Aligned
processing SVL_C1_S192_P384_VC1_003696_005031_Arousal_AV_Aligned
processing SAH_C1_S028_P055_VC1_003774_004919_Arousal_AV_Aligned
processing SVH_C1_S028_P055_VC1_000001_000801_Arousal_AV_Aligned
processing SSL_C1_S018_P035_VC1_002201_002701_Arousal_AV_Aligned
processing SSD_C1_S023_P046_VC1_003001_003501_Arousal_AV_Aligned
processing SVL_C1_S014_P027_VC1_004045_006075_Arousal_AV_Aligned
processing

In [6]:
print(f'{train_inputs[0].size()=}')
print(f'{train_labels[0].size()=}')

# 计算第一个维度的总长度
total_length = sum(tensor.size(0) for tensor in train_inputs)
# 计算平均长度
average_length = total_length // len(train_inputs)


# 提取每个张量的第一个维度长度
lengths = [tensor.size(0) for tensor in train_inputs]

# 对长度进行排序
sorted_lengths = sorted(lengths)
median_length=sorted_lengths[len(sorted_lengths)//2]
max_len=sorted_lengths[-1]

print(f'{average_length=}_{median_length=}_{max_len=}')

train_inputs[0].size()=torch.Size([801, 35])
train_labels[0].size()=torch.Size([801, 1])
average_length=1055_median_length=907_max_len=2801


## Model

In [7]:
class EmbeddingLayer(nn.Module):
    def __init__(self, input_dim, embedding_dim):
        super(EmbeddingLayer, self).__init__()
        self.fc = nn.Linear(input_dim, embedding_dim)

    def forward(self, x):
        # x:(batch_size,feature_dim)
        x = self.fc(x)
        return x

## Transformer

In [8]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=0.1)
        
        # Initialize positional encoding matrix
        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        
        self.register_buffer('pe', pe)

    def forward(self, x):
        # Add positional encoding to the input tensor
        x = x + self.pe[:, :x.size(1)]
        return self.dropout(x)


class RegressionTransformer(nn.Module):
    def __init__(self, args):
        super(RegressionTransformer, self).__init__()
        self.args = args
        # embed_dim = head_dim * num_heads?
        self.embedding = nn.Linear(args['input_size'], args['d_model'])
        self.output_fc = nn.Linear(args['input_size'], args['d_model'])
        self.position_encoding = PositionalEncoding(args['d_model'], args['aligned_len'])  # Positional encoding layer
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=args['d_model'],
            nhead=8,
            dim_feedforward=4 * args['d_model'],
            batch_first=True,
            dropout=0.1,
            device=args['device']
        )
        decoder_layer = nn.TransformerDecoderLayer(
            d_model=args['d_model'],
            nhead=8,
            dropout=0.1,
            dim_feedforward=4 * args['d_model'],
            batch_first=True,
            device=args['device']
        )
        self.encoder = torch.nn.TransformerEncoder(encoder_layer, num_layers=5)
        self.decoder = torch.nn.TransformerDecoder(decoder_layer, num_layers=5)
        self.fc = nn.Linear(args['d_model'], args['output_size'])
        self.fc1 = nn.Linear(args['aligned_len'] * args['d_model'], args['d_model'])
        self.fc2 = nn.Linear(args['d_model'], args['output_size'])

        self.aligned_len = args['aligned_len']
        self.input_atten_mask = self._generate_square_subsequent_mask(args['aligned_len'])
        
    def forward(self, x, y, input_mask, tgt_mask):
    
        input_atten_mask = self.input_atten_mask # shape(batchsize,seqlen,seqlen)
        has_nan = torch.isnan(x).any()
        # print(f'loc0{has_nan=}')
        # x shape(batchsize,seqlen,input_size)
        x = self.embedding(x) # x shape(batchsize,seqlen,d_model)
        has_nan = torch.isnan(x).any()
        # print(f'loc1{has_nan=}')
        x = self.position_encoding(x)  # x shape(batchsize,seqlen,d_model)
        has_nan = torch.isnan(x).any()
        # print(f'loc2{has_nan=}')
        x = self.encoder(src=x, mask=input_atten_mask, src_key_padding_mask=input_mask) # x shape(batchsize,seqlen,d_model)
        has_nan = torch.isnan(x).any()
        # print(f'loc3{has_nan=}')
        # print(f'{y.shape=}_{x.shape=}')
        extend_y=y.expand(-1, -1, self.args['d_model'])
        x = self.decoder(tgt=extend_y, memory=x, tgt_mask=input_atten_mask) # x shape(batchsize,seqlen,d_model)
        has_nan = torch.isnan(x).any()
        # print(f'loc4{has_nan=}')
        x = self.fc(x) # x shape(batchsize,seqlen,1)
        has_nan = torch.isnan(x).any()
        # print(f'loc5{has_nan=}')
        return x

    def _generate_square_subsequent_mask(self, size):
        mask = (torch.triu(torch.ones(size, size)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        mask = mask.bool()
        mask = mask.to(self.args['device'])
        return mask


In [9]:
def generate_square_subsequent_mask(size):
        mask = (torch.triu(torch.ones(size, size)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        mask = mask.bool()
        return mask
    
    
size = 6
mask = generate_square_subsequent_mask(size)
print("Test Case 1:")
print(mask)


Test Case 1:
tensor([[False,  True,  True,  True,  True,  True],
        [False, False,  True,  True,  True,  True],
        [False, False, False,  True,  True,  True],
        [False, False, False, False,  True,  True],
        [False, False, False, False, False,  True],
        [False, False, False, False, False, False]])


In [10]:
class SEWADataset(Dataset):
    """
    Dataset definition

    """
    def __init__(self, dataset, labels, maxlen, transform=None):
        self.dataset=dataset
        self.labels = labels  # Added feature_type argument
        self.transform = transform
        self.maxlen=maxlen

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        feature=self.dataset[idx]
        label = self.labels[idx]


         # Convert feature to torch.float32
        feature = torch.tensor(feature, dtype=torch.float32)
        # Convert label to torch.float32
        label = torch.tensor(label, dtype=torch.float32)
        
        
        pad_len = self.maxlen - feature.shape[0]
        if pad_len > 0:
            # print(f'before padding{feature.size()=}')
            feature = torch.nn.functional.pad(feature, (0, 0, 0, pad_len), value=0)
            # print(f'before padding{feature.size()=}')
            # print(f'before padding{label.size()=}')
            # label = torch.nn.functional.pad(label, (0,pad_len), value=0)  # Assuming -1 represents padding for labels
            padding_length = self.maxlen - label.shape[0]
            padding = torch.zeros(padding_length, 1)
            label = torch.cat((label, padding), dim=0)

            # print(f'after padding {label.size()=}')

        if pad_len < 0:
            
            feature = feature[:self.maxlen]
            label = label[:self.maxlen]
        
        
        # Generate feature and label masks
        feature_mask = torch.zeros(self.maxlen)
        label_mask = torch.zeros(self.maxlen)
        if pad_len > 0:
            feature_mask[pad_len:] = 1  # Set mask to 1 for padded positions
            label_mask[pad_len:] = 1  # Set mask to 1 for padded positions
        

        if self.transform:
            feature = self.transform(feature)

        return feature, label,feature_mask,label_mask
    def get_all_labels(self):
            return self.labels

In [11]:
class SEWAdataLoader(DataLoader):
    """
    
    """
    def __init__(self, dataset, batch_size=1, shuffle=False, num_workers=0):
        super(SEWAdataLoader, self).__init__(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)

      
            
    def __iter__(self):

                for idx in range(0, len(self.dataset), self.batch_size):
                    # get current mini-batch sample
                    batch_data = []
                    for i in range(idx, min(idx + self.batch_size, len(self.dataset))):
                        batch_data.append(self.dataset[i])

                    # Obtain the features and labels of each sample in the current mini-batch
                    batch_inputs = [sample[0] for sample in batch_data]
                    batch_labels = [sample[1] for sample in batch_data]
                    batch_inputs_mask=[sample[2] for sample in batch_data]
                    batch_labels_mask=[sample[3] for sample in batch_data]

                    # print(f'{batch_inputs[0].size()=}_{batch_labels[0].size()=}_{batch_inputs_mask[0].size()=}_{batch_labels_mask[0].size()=}')
                    
                    
                    
                    tensor_inputs = torch.stack(batch_inputs, dim=0)
                    tensor_labels = torch.stack(batch_labels, dim=0)
                    tensor_inputs_mask = torch.stack(batch_inputs_mask, dim=0)
                    tensor_labels_mask = torch.stack(batch_labels_mask, dim=0)
                    # print(f'{tensor_inputs.shape=}')
                    yield tensor_inputs, tensor_labels, tensor_inputs_mask, tensor_labels_mask
                    

## Training module

In [12]:

def train_model(model, train_loader, test_loader, criterion, optimizer, num_epochs,device):
    for epoch in range(num_epochs):
        # Training the model
        model = model.to(device)
        model.train()
        for batch_inputs, batch_labels,batch_inputs_mask,batch_labels_mask in train_loader:
            # Passing input data x and labels y into the model
            batch_inputs = batch_inputs.to(device)
            batch_labels = batch_labels.to(device)
            batch_inputs_mask = batch_inputs_mask.to(device)
            batch_labels_mask = batch_labels_mask.to(device)
            outputs = model(batch_inputs,batch_labels,batch_inputs_mask,batch_labels_mask)
            
            
            flipped_labels_mask = (batch_labels_mask == 0)
            
            masked_outputs = outputs * flipped_labels_mask.float().unsqueeze(-1)
            
            
            # masked_outputs = outputs * batch_labels_mask.unsqueeze(-1)
            # print(f'{masked_outputs=}')
            # 找到每个序列中的最小值和最大值
            min_vals, _ = torch.min(masked_outputs, dim=1, keepdim=True)  # 在 seq_len 维度上找到最小值
            max_vals, _ = torch.max(masked_outputs, dim=1, keepdim=True)  # 在 seq_len 维度上找到最大值

            # 将每个元素减去最小值，然后除以最大值与最小值之差
            normalized_outputs = (masked_outputs - min_vals) / (max_vals - min_vals)
            
            masked_labels = batch_labels * flipped_labels_mask.float().unsqueeze(-1)
            # print(f'{normalized_outputs=}')
            # print(f'{masked_labels=}')
            loss = criterion(normalized_outputs, masked_labels)
            print(f'{loss=}')
            # Backpropagation and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
          # Printing the loss for the current iteration

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
        
        
    # Evaluating the model's performance
    model.eval()
    all_predicted = []
    all_labels = []
    total_loss = 0.0
    total_samples = 0

    for batch_inputs, batch_labels in test_loader:
       
        # Forward propagation
        outputs = model(batch_inputs)
        loss = criterion(outputs, batch_labels)
        total_loss += loss.item() * batch_inputs.size(0)
        total_samples += batch_inputs.size(0)

        # Calculating predicted values and true labels
        all_predicted.extend(outputs.tolist())
        all_labels.extend(batch_labels.tolist())

    # Calculating MSE, MAE and R2 score
    mse = mean_squared_error(all_labels, all_predicted)
    mae = mean_absolute_error(all_labels, all_predicted)
    r2 = r2_score(all_labels, all_predicted)

    print(f'Mean Squared Error (MSE): {mse:.4f}, Mean Absolute Error (MAE): {mae:.4f}, R-squared (R2): {r2:.4f}')
    return mse, mae, r2, all_predicted, all_labels


In [13]:

args = {
    'input_size': 35,     # 每个帧的特征向量维度
    'd_model': 64,        # Transformer隐藏单元数
    'num_layers': 4,      # Encoder/Decoder层数
    'output_size': 2,     # 输出的维度
    'device': 'cuda',      # 训练设备
    'aligned_len': 907,  # 对齐后的序列长度
    'num_epochs': 20      # 训练的轮数
}

# 初始化模型
model = RegressionTransformer(args)
# 定义损失函数和优化器
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

traindataset=SEWADataset(dataset=train_inputs,labels=train_labels,maxlen=args['aligned_len'])
train_loader=SEWAdataLoader(traindataset,batch_size=16)
testdataset=SEWADataset(dataset=val_inputs,labels=val_labels,maxlen=args['aligned_len'])
test_loader=SEWAdataLoader(testdataset,batch_size=16)
mse, mae, r2,all_predicted,all_labels=train_model(model, train_loader, test_loader, criterion, optimizer, args['num_epochs'],args['device'])

print(f'{mse=}_{mae=}_{r2=}')


/tmp/ipykernel_23487/3039105427.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  feature = torch.tensor(feature, dtype=torch.float32)
/tmp/ipykernel_23487/3039105427.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(label, dtype=torch.float32)
/home/ubuntu/anaconda3/envs/hr-viton/lib/python3.8/site-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([16, 907, 1])) that is different to the input size (torch.Size([16, 907, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


batch_inputs[0].size()=torch.Size([907, 35])_batch_labels[0].size()=torch.Size([907, 1])_batch_inputs_mask[0].size()=torch.Size([907])_batch_labels_mask[0].size()=torch.Size([907])
loss=tensor(0.4751, device='cuda:0', grad_fn=<MseLossBackward0>)
batch_inputs[0].size()=torch.Size([907, 35])_batch_labels[0].size()=torch.Size([907, 1])_batch_inputs_mask[0].size()=torch.Size([907])_batch_labels_mask[0].size()=torch.Size([907])
loss=tensor(0.1887, device='cuda:0', grad_fn=<MseLossBackward0>)
batch_inputs[0].size()=torch.Size([907, 35])_batch_labels[0].size()=torch.Size([907, 1])_batch_inputs_mask[0].size()=torch.Size([907])_batch_labels_mask[0].size()=torch.Size([907])
loss=tensor(0.1643, device='cuda:0', grad_fn=<MseLossBackward0>)
batch_inputs[0].size()=torch.Size([907, 35])_batch_labels[0].size()=torch.Size([907, 1])_batch_inputs_mask[0].size()=torch.Size([907])_batch_labels_mask[0].size()=torch.Size([907])
loss=tensor(0.1142, device='cuda:0', grad_fn=<MseLossBackward0>)
batch_inputs[0].

/home/ubuntu/anaconda3/envs/hr-viton/lib/python3.8/site-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([1, 907, 1])) that is different to the input size (torch.Size([1, 907, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


batch_inputs[0].size()=torch.Size([907, 35])_batch_labels[0].size()=torch.Size([907, 1])_batch_inputs_mask[0].size()=torch.Size([907])_batch_labels_mask[0].size()=torch.Size([907])
loss=tensor(0.1307, device='cuda:0', grad_fn=<MseLossBackward0>)
batch_inputs[0].size()=torch.Size([907, 35])_batch_labels[0].size()=torch.Size([907, 1])_batch_inputs_mask[0].size()=torch.Size([907])_batch_labels_mask[0].size()=torch.Size([907])
loss=tensor(0.1218, device='cuda:0', grad_fn=<MseLossBackward0>)
batch_inputs[0].size()=torch.Size([907, 35])_batch_labels[0].size()=torch.Size([907, 1])_batch_inputs_mask[0].size()=torch.Size([907])_batch_labels_mask[0].size()=torch.Size([907])
loss=tensor(0.0895, device='cuda:0', grad_fn=<MseLossBackward0>)
batch_inputs[0].size()=torch.Size([907, 35])_batch_labels[0].size()=torch.Size([907, 1])_batch_inputs_mask[0].size()=torch.Size([907])_batch_labels_mask[0].size()=torch.Size([907])
loss=tensor(0.1163, device='cuda:0', grad_fn=<MseLossBackward0>)
batch_inputs[0].

ValueError: too many values to unpack (expected 2)